# Chapter 41: Database CI/CD

Database schema changes present unique challenges in continuous delivery pipelines. Unlike application code, which can be rolled back by redeploying a previous container image, database schema modifications are persistent and often irreversible. A dropped column cannot be restored by rolling back application code, and schema changes that lock tables for extended periods can cause production outages. Furthermore, microservices architectures mandate that each service owns its data, requiring coordination between application deployment and database migration timing.

This chapter establishes patterns for safely evolving database schemas within CI/CD pipelines, ensuring that application deployments and schema changes remain compatible across versions. We examine migration-based schema management, testing strategies for database changes, backward compatibility patterns that enable zero-downtime deployments, and rollback procedures for when migrations fail.

## 41.1 Database Migration Strategies

Database migrations transform schema from one version to another while preserving data. Unlike application code deployments that replace binaries, migrations modify persistent state that must remain available throughout the deployment process.

### Migration-Based Schema Management

Migration tools track schema versions in metadata tables, ensuring each change executes exactly once across all environments.

**Flyway Migration Structure:**
```
database/migrations/
├── V1__Initial_schema.sql
├── V2__Add_user_preferences.sql
├── V3__Create_payment_indexes.sql
├── V4__Add_order_status.sql
└── U4__Rollback_order_status.sql  # Undo migration
```

**Naming Convention Explanation:**
- `V{version}__{description}.sql`: Versioned migrations executed sequentially
- `U{version}__{description}.sql`: Undo migrations for rollback (Flyway Teams/Enterprise)
- `R__{description}.sql`: Repeatable migrations executed whenever checksum changes (views, stored procedures)
- Prefixes configurable: `V` (versioned), `U` (undo), `R` (repeatable)

**V1__Initial_schema.sql:**
```sql
CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    email VARCHAR(255) NOT NULL UNIQUE,
    created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
);

CREATE INDEX idx_users_email ON users(email);

-- Flyway metadata table created automatically
-- flyway_schema_history tracks applied migrations
```

**V2__Add_user_preferences.sql:**
```sql
ALTER TABLE users 
ADD COLUMN preferences JSONB DEFAULT '{}';

COMMENT ON COLUMN users.preferences IS 'User preference settings in JSON format';
```

**Explanation:**
Flyway executes migrations in version order (V1, then V2). The `flyway_schema_history` table records which migrations have been applied, preventing re-execution on subsequent starts. The JSONB type in PostgreSQL provides efficient storage and querying of semi-structured data, with the default empty object ensuring existing rows remain valid.

### Flyway Configuration

```yaml
# flyway.conf
flyway.url=jdbc:postgresql://localhost:5432/payments
flyway.user=flyway_user
flyway.password=${DB_PASSWORD}
flyway.schemas=public
flyway.defaultSchema=public

-- Migration behavior
flyway.baselineOnMigrate=true           -- Baseline existing databases
flyway.validateOnMigrate=true           -- Validate checksums before migrating
flyway.cleanDisabled=true               -- Prevent accidental data loss in production
flyway.outOfOrder=false                 -- Strict version ordering

-- Placeholders for environment-specific values
flyway.placeholders.schemaOwner=payment_service
flyway.placeholders.tablespace=pg_default

-- Callbacks (hooks for lifecycle events)
flyway.callbacks=beforeMigrate.sql,afterMigrate.sql
```

**Spring Boot Integration:**
```yaml
# application.yml
spring:
  flyway:
    enabled: true
    locations: classpath:db/migration
    baseline-on-migrate: true
    validate-on-migrate: true
    clean-disabled: true  # Critical for production safety
    
    # Environment-specific configuration
    placeholders:
      schema_owner: ${DB_SCHEMA_OWNER:payment_service}
      partition_suffix: ${DB_PARTITION_SUFFIX:}
    
    # Retry configuration for CI/CD
    connect-retries: 10
    connect-retries-interval: 5s
    
  datasource:
    url: jdbc:postgresql://${DB_HOST:localhost}:5432/payments
    username: ${DB_USER:payment_service}
    password: ${DB_PASSWORD}
    hikari:
      maximum-pool-size: 5
      connection-timeout: 30000
```

**Explanation:**
The configuration disables `clean` (which drops all objects) in production to prevent catastrophic data loss. Placeholders allow SQL parameterization for environment-specific values (schema owners, tablespaces). The `validateOnMigrate` ensures migration files haven't been modified after application—Flyway calculates checksums and fails if a previously applied migration changed, preventing drift.

### Liquibase Alternative

Liquibase uses XML, YAML, or JSON changelogs for database-agnostic migrations:

```xml
<!-- db/changelog/db.changelog-master.xml -->
<?xml version="1.0" encoding="UTF-8"?>
<databaseChangeLog
    xmlns="http://www.liquibase.org/xml/ns/dbchangelog"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://www.liquibase.org/xml/ns/dbchangelog
                        http://www.liquibase.org/xml/ns/dbchangelog/dbchangelog-4.20.xsd">
    
    <changeSet id="1" author="alice">
        <createTable tableName="users">
            <column name="id" type="SERIAL">
                <constraints primaryKey="true"/>
            </column>
            <column name="email" type="VARCHAR(255)">
                <constraints nullable="false" unique="true"/>
            </column>
            <column name="created_at" type="TIMESTAMP" 
                    defaultValueComputed="CURRENT_TIMESTAMP">
                <constraints nullable="false"/>
            </column>
        </createTable>
        
        <createIndex indexName="idx_users_email" 
                     tableName="users">
            <column name="email"/>
        </createIndex>
        
        <rollback>
            <dropTable tableName="users"/>
        </rollback>
    </changeSet>
    
    <changeSet id="2" author="bob">
        <addColumn tableName="users">
            <column name="preferences" type="JSONB" 
                    defaultValue="{}"/>
        </addColumn>
        
        <rollback>
            <dropColumn tableName="users" 
                       columnName="preferences"/>
        </rollback>
    </changeSet>
</databaseChangeLog>
```

**Explanation:**
Liquibase tracks changes via `databasechangelog` table. Each `changeSet` has an ID and author, forming a composite key. The `rollback` tags define how to reverse changes (used by `liquibase rollback` commands). XML format provides better database abstraction than SQL—Liquibase translates `JSONB` to appropriate types for PostgreSQL, MySQL, or Oracle automatically.

## 41.2 Schema Versioning

Schema versioning ensures that application version N is compatible with database schema version M, enabling safe rollbacks and blue-green deployments.

### Version Compatibility Matrix

| Application Version | Database Schema Version | Compatibility |
|---------------------|------------------------|---------------|
| 2.1.0 | 15 | Full |
| 2.0.9 | 14 | Read-only (backward compatible) |
| 2.1.0 | 14 | Forward compatible (new code, old schema) |

**Application Startup Check:**
```java
@Component
public class SchemaVersionChecker implements CommandLineRunner {
    
    @Autowired
    private JdbcTemplate jdbcTemplate;
    
    @Value("${app.min-schema-version:15}")
    private int minSchemaVersion;
    
    @Override
    public void run(String... args) {
        Integer currentVersion = jdbcTemplate.queryForObject(
            "SELECT MAX(version) FROM flyway_schema_history WHERE success = true",
            Integer.class
        );
        
        if (currentVersion == null || currentVersion < minSchemaVersion) {
            throw new IllegalStateException(
                String.format("Schema version %d is below minimum required %d. " +
                    "Please run database migrations before starting application.",
                    currentVersion, minSchemaVersion)
            );
        }
        
        log.info("Schema version {} validated (minimum required: {})", 
                 currentVersion, minSchemaVersion);
    }
}
```

**Explanation:**
The checker queries Flyway's history table on startup, verifying the database schema meets the application's minimum requirements. This prevents application crashes from missing columns or tables, failing fast during deployment rather than at runtime when features access non-existent schema elements.

### Migration Versioning Strategy

**Timestamp vs. Sequential:**
```bash
# Sequential (Flyway default)
V1__create_users.sql
V2__add_preferences.sql

# Timestamp (useful for parallel development)
V20240115120000__create_users.sql
V20240115123000__add_preferences.sql
```

**Best Practice:**
Use sequential numbers for small teams, timestamps for large teams where multiple developers might create migrations simultaneously. Timestamps prevent version conflicts when merging feature branches.

## 41.3 Testing Database Changes

Database migrations require testing in ephemeral environments that mirror production schema characteristics.

### Testcontainers for Integration Testing

```java
@SpringBootTest
@Testcontainers
public class DatabaseMigrationTest {
    
    @Container
    static PostgreSQLContainer<?> postgres = new PostgreSQLContainer<>(
        DockerImageName.parse("postgres:15-alpine"))
        .withDatabaseName("testdb")
        .withUsername("test")
        .withPassword("test")
        .withInitScript("db/init.sql");  // Baseline schema
    
    @DynamicPropertySource
    static void configureProperties(DynamicPropertyRegistry registry) {
        registry.add("spring.datasource.url", postgres::getJdbcUrl);
        registry.add("spring.datasource.username", postgres::getUsername);
        registry.add("spring.datasource.password", postgres::getPassword);
    }
    
    @Autowired
    private Flyway flyway;
    
    @Autowired
    private JdbcTemplate jdbcTemplate;
    
    @Test
    void testMigrationsRunSuccessfully() {
        // Trigger migrations
        flyway.migrate();
        
        // Verify schema version
        assertEquals(4, flyway.info().current().getVersion().getMajor());
        
        // Verify data integrity after migration
        Integer userCount = jdbcTemplate.queryForObject(
            "SELECT COUNT(*) FROM users", Integer.class);
        assertNotNull(userCount);
    }
    
    @Test
    void testNewColumnAccessible() {
        // Arrange: Insert user with preferences
        jdbcTemplate.update(
            "INSERT INTO users (email, preferences) VALUES (?, ?::jsonb)",
            "test@example.com", 
            "{\"theme\": \"dark\"}"
        );
        
        // Act & Assert: Query JSONB column
        String theme = jdbcTemplate.queryForObject(
            "SELECT preferences->>'theme' FROM users WHERE email = ?",
            String.class, 
            "test@example.com"
        );
        
        assertEquals("dark", theme);
    }
    
    @Test
    void testRollbackCapability() {
        // Apply migrations
        flyway.migrate();
        
        // Verify rollback works (if using Flyway Teams)
        flyway.undo();
        
        // Verify schema reverted
        assertThrows(BadSqlGrammarException.class, () -> {
            jdbcTemplate.queryForObject(
                "SELECT preferences FROM users LIMIT 1", 
                String.class);
        });
    }
}
```

**Explanation:**
Testcontainers spins up a real PostgreSQL container for tests. `@DynamicPropertySource` injects the container's connection details into Spring's environment. Tests verify:
1. Migrations apply cleanly (no SQL errors)
2. Schema version matches expected
3. New schema features work (JSONB querying)
4. Rollback restores previous state (for supported databases)

### Migration Performance Testing

Test long-running migrations in staging:

```java
@Test
void testLargeTableMigrationPerformance() {
    // Populate with production-like volume
    jdbcTemplate.update(
        "INSERT INTO transactions (amount, status) " +
        "SELECT random() * 1000, 'pending' " +
        "FROM generate_series(1, 1000000)"
    );
    
    long startTime = System.currentTimeMillis();
    
    // Run migration that adds index
    jdbcTemplate.update(
        "CREATE INDEX CONCURRENTLY idx_transactions_status " +
        "ON transactions(status)"
    );
    
    long duration = System.currentTimeMillis() - startTime;
    
    // Assert completion within maintenance window (5 minutes)
    assertTrue(duration < 300000, 
        "Migration took too long: " + duration + "ms");
}
```

**Explanation:**
This test verifies that index creation on a million-row table completes within the maintenance window. `CREATE INDEX CONCURRENTLY` (PostgreSQL) allows reads/writes during index building, preventing table locks, but takes longer than standard `CREATE INDEX`.

## 41.4 Database Per Service Pattern

Microservices architectures require each service to own its data, accessed only via the service's API.

### Schema Isolation

```yaml
# docker-compose.yml for local development
version: '3.8'
services:
  payment-service:
    build: ./services/payment
    environment:
      - DB_HOST=payment-db
      - DB_NAME=payments
      - DB_SCHEMA=payment_service
  
  payment-db:
    image: postgres:15
    environment:
      POSTGRES_DB: payments
      POSTGRES_USER: payment_service
      POSTGRES_PASSWORD: secure_password
    volumes:
      - payment-data:/var/lib/postgresql/data
      - ./services/payment/migrations:/docker-entrypoint-initdb.d
  
  order-service:
    build: ./services/order
    environment:
      - DB_HOST=order-db
      - DB_NAME=orders
  
  order-db:
    image: postgres:15
    environment:
      POSTGRES_DB: orders
      POSTGRES_USER: order_service
    volumes:
      - order-data:/var/lib/postgresql/data

volumes:
  payment-data:
  order-data:
```

**Explanation:**
Each service has a dedicated database container in development, enforcing the "database per service" pattern. Services cannot accidentally query each other's tables because they're physically separated. This mirrors production where services might use separate RDS instances or schemas.

### Shared Database Anti-Pattern Prevention

Prevent services from sharing databases in production:

```yaml
# Kubernetes NetworkPolicy
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: payment-db-access
  namespace: databases
spec:
  podSelector:
    matchLabels:
      app: payment-postgres
  policyTypes:
    - Ingress
  ingress:
    - from:
        - namespaceSelector:
            matchLabels:
              name: payment
          podSelector:
            matchLabels:
              app: payment-service
      ports:
        - protocol: TCP
          port: 5432
```

**Explanation:**
This NetworkPolicy allows only the payment-service Pod to connect to the payment database on port 5432. The order-service cannot access the payment database even if it has the credentials, enforcing architectural boundaries at the network layer.

## 41.5 Backward Compatibility

Zero-downtime deployments require database schemas that support both old and new application versions simultaneously.

### Expand-Contract Pattern

The expand-contract pattern phases schema changes across multiple deployments:

**Phase 1: Expand (Deploy 1)**
```sql
-- V10__add_user_status.sql
ALTER TABLE users 
ADD COLUMN status VARCHAR(20) DEFAULT 'active';

-- Backfill existing data
UPDATE users 
SET status = 'active' 
WHERE status IS NULL;

-- Create index concurrently (no table lock)
CREATE INDEX CONCURRENTLY idx_users_status 
ON users(status);

-- Add constraint after data is clean
ALTER TABLE users 
ALTER COLUMN status SET NOT NULL;
```

**Application Code (Version 2.0):**
```java
@Entity
@Table(name = "users")
public class User {
    @Id
    private Long id;
    
    @Column(nullable = false)
    private String email;
    
    // New field added, but old code ignores it
    @Column(nullable = false)
    private String status = "active";  // Default for backward compatibility
    
    // Getters and setters
}
```

**Phase 2: Migrate Data (Deploy 2)**
```sql
-- V11__migrate_status_data.sql
-- Optional: data transformation if needed
UPDATE users 
SET status = UPPER(status);
```

**Phase 3: Contract (Deploy 3)**
```sql
-- V12__remove_old_status.sql (weeks later)
-- Only after all old app instances stopped
-- ALTER TABLE users DROP COLUMN old_status;
```

**Explanation:**
- **Expand**: Add new column with defaults, ensuring old code continues working (ignores new column)
- **Contract**: Remove old columns only after new code is fully deployed and stable
- **Concurrent Indexes**: `CREATE INDEX CONCURRENTLY` prevents table locks during index creation, allowing reads/writes to continue

### Blue-Green Safe Schema Changes

```sql
-- Safe: Add nullable column
ALTER TABLE orders 
ADD COLUMN tracking_number VARCHAR(100) NULL;

-- Safe: Create index concurrently
CREATE INDEX CONCURRENTLY idx_orders_tracking 
ON orders(tracking_number);

-- Safe: Add table with foreign key (new table, no existing data)
CREATE TABLE order_tracking (
    id SERIAL PRIMARY KEY,
    order_id INTEGER REFERENCES orders(id),
    tracking_data JSONB
);

-- Unsafe: Drop column (breaks old app version)
-- ALTER TABLE orders DROP COLUMN legacy_field;

-- Unsafe: Rename column (breaks queries)
-- ALTER TABLE orders RENAME COLUMN total TO order_total;

-- Unsafe: Add non-nullable column without default
-- ALTER TABLE orders ADD COLUMN priority INTEGER NOT NULL;
```

**Explanation:**
Adding nullable columns and new tables is always safe because existing code doesn't reference them. Dropping or renaming columns breaks old application versions still running during deployment. Adding non-nullable columns without defaults fails on existing rows.

## 41.6 Handling Long-Running Transactions

Schema changes on large tables can lock tables for hours, causing downtime.

### Online Schema Changes

**pt-online-schema-change (Percona Toolkit):**
```bash
# Safely alter large table without locking
pt-online-schema-change \
    --alter "ADD COLUMN preferences JSONB DEFAULT '{}'" \
    --execute \
    --max-load "Threads_running=25" \
    --critical-load "Threads_running=50" \
    --chunk-size 1000 \
    --progress percentage,10 \
    D=payments,t=users,u=admin,p=password
```

**Explanation:**
`pt-online-schema-change` creates a new table with the altered schema, copies data in chunks (1000 rows at a time), tracks changes via triggers, and swaps tables atomically. The `--max-load` parameter pauses copying if server load exceeds thresholds, preventing performance degradation.

**gh-ost (GitHub Online Schema Transformer):**
```bash
gh-ost \
  --database=payments \
  --table=users \
  --alter="ADD COLUMN preferences JSONB DEFAULT '{}'" \
  --execute \
  --max-load=Threads_running=25 \
  --chunk-size=1000 \
  --throttle-control-replicas="replica1,replica2" \
  --initially-drop-ghost-table \
  --initially-drop-old-table
```

**Explanation:**
`gh-ost` uses binary log streaming instead of triggers (less overhead than pt-online-schema-change), throttles based on replication lag, and provides progress feedback. It works by creating a "ghost" table, streaming changes from the binlog, and cutting over via atomic rename.

### PostgreSQL Specifics

```sql
-- PostgreSQL supports transactional DDL
BEGIN;
ALTER TABLE users 
ADD COLUMN preferences JSONB DEFAULT '{}';
CREATE INDEX CONCURRENTLY idx_users_prefs ON users(preferences);
COMMIT;

-- But CONCURRENTLY cannot run inside transaction
-- Run separately:
CREATE INDEX CONCURRENTLY idx_users_prefs ON users(preferences);
```

## 41.7 Rollback Strategies

When migrations fail, recovery procedures must restore service quickly.

### Application-Level Rollback

If schema change is backward compatible, roll back application only:

```bash
# Kubernetes rollback
kubectl rollout undo deployment/payment-service

# Helm rollback
helm rollback payment-service 2  # Revision 2
```

**Database remains at new schema version**, but old application code works because schema was expanded (added nullable columns).

### Database Rollback

For failed migrations, use undo scripts:

```sql
-- U5__rollback_add_preferences.sql (Flyway undo)
ALTER TABLE users DROP COLUMN preferences;
```

**Execution:**
```bash
# Flyway Teams/Enterprise
flyway undo -target=4

# Or manual rollback
psql -U admin -d payments -f manual_rollback.sql
```

### Backup Before Migration

```yaml
apiVersion: batch/v1
kind: Job
metadata:
  name: pre-migration-backup
spec:
  template:
    spec:
      containers:
      - name: backup
        image: postgres:15
        command:
        - sh
        - -c
        - |
          pg_dump $DATABASE_URL \
            --clean --if-exists \
            --no-owner \
            --no-privileges \
            > /backups/pre-migration-$(date +%s).sql
        env:
        - name: DATABASE_URL
          valueFrom:
            secretKeyRef:
              name: db-credentials
              key: url
        volumeMounts:
        - name: backups
          mountPath: /backups
      volumes:
      - name: backups
        persistentVolumeClaim:
          claimName: db-backups
      restartPolicy: Never
```

**Explanation:**
The Job runs `pg_dump` before migrations, creating a timestamped backup. If migration corrupts data, restore from this backup:

```bash
psql -d payments -f /backups/pre-migration-1705312800.sql
```

## 41.8 CI/CD Integration

### Pipeline Stages

```yaml
# .github/workflows/database-migration.yml
name: Database CI/CD
on:
  push:
    paths:
      - 'database/migrations/**'
      - 'src/main/resources/db/migration/**'

jobs:
  validate:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Validate Migration Naming
        run: |
          for file in database/migrations/*.sql; do
            if [[ ! $file =~ ^database/migrations/V[0-9]+__.*\.sql$ ]]; then
              echo "Invalid migration name: $file"
              exit 1
            fi
          done
      
      - name: Check for Forbidden Operations
        run: |
          if grep -r "DROP TABLE\|DROP COLUMN" database/migrations/; then
            echo "ERROR: Destructive changes detected. Use expand-contract pattern."
            exit 1
          fi

  test:
    runs-on: ubuntu-latest    services:
      postgres:
        image: postgres:15
        env:
          POSTGRES_PASSWORD: postgres
        options: >-
          --health-cmd pg_isready
          --health-interval 10s
          --health-timeout 5s
          --health-retries 5
        ports:
          - 5432:5432
    
    steps:
      - uses: actions/checkout@v4
      
      - name: Setup Java
        uses: actions/setup-java@v3
        with:
          java-version: '17'
          distribution: 'temurin'
      
      - name: Run Migrations
        run: |
          ./mvnw flyway:migrate \
            -Dflyway.url=jdbc:postgresql://localhost:5432/postgres \
            -Dflyway.user=postgres \
            -Dflyway.password=postgres
      
      - name: Test Application
        run: ./mvnw test

  deploy-staging:
    needs: [validate, test]
    runs-on: ubuntu-latest
    environment: staging
    steps:
      - name: Run Migration
        run: |
          kubectl create job migration-$(date +%s) \
            --from=cronjob/db-migrate \
            -n staging

  deploy-production:
    needs: deploy-staging
    runs-on: ubuntu-latest
    environment: production  # Requires approval
    steps:
      - name: Backup Database
        run: kubectl apply -f k8s/backup-job.yaml
      
      - name: Run Migration
        run: kubectl create job migration-$(date +%s) \
            --from=cronjob/db-migrate \
            -n production
      
      - name: Verify Migration
        run: |
          kubectl wait --for=condition=complete \
            job/migration-* --timeout=300s -n production
```

**Explanation:**
The pipeline validates migration naming conventions (V{number}__{description}.sql), checks for destructive operations (DROP statements), tests migrations against PostgreSQL in CI, requires staging validation before production, creates backups before production migrations, and verifies job completion before proceeding.

### Migration Container Pattern

```dockerfile
# Dockerfile.migrate
FROM flyway/flyway:9-alpine

COPY database/migrations /flyway/sql

ENV FLYWAY_CONNECT_RETRIES=10
ENV FLYWAY_BASELINE_ON_MIGRATE=true

ENTRYPOINT ["flyway"]
CMD ["migrate"]
```

**Kubernetes Job:**
```yaml
apiVersion: batch/v1
kind: Job
metadata:
  name: db-migrate
spec:
  template:
    spec:
      containers:
      - name: migrate
        image: company/db-migrate:v2.1.0
        env:
        - name: FLYWAY_URL
          valueFrom:
            secretKeyRef:
              name: db-credentials
              key: url
        - name: FLYWAY_USER
          valueFrom:
            secretKeyRef:
              name: db-credentials
              key: username
        - name: FLYWAY_PASSWORD
          valueFrom:
            secretKeyRef:
              name: db-credentials
              key: password
      restartPolicy: Never
  backoffLimit: 3
```

**Explanation:**
Migrations run as Kubernetes Jobs using the Flyway image with embedded SQL scripts. The Job retries up to 3 times on failure. This ensures migrations complete before the application Deployment updates, using Helm hooks or ArgoCD sync waves to enforce ordering.

---

## Chapter Summary and Preview

This chapter addressed the critical intersection of database schema management and continuous delivery, establishing patterns for safely evolving persistent data structures without service interruption. We examined migration-based schema management using Flyway and Liquibase, which track applied changes in metadata tables to ensure idempotent, ordered execution across environments. The expand-contract pattern emerged as the fundamental strategy for zero-downtime schema changes, separating the addition of new structures (expand) from the removal of old structures (contract) across multiple deployment cycles, ensuring backward compatibility between application versions and database schemas.

Testing strategies using Testcontainers provide confidence that migrations execute correctly and performantly against production-like data volumes, while online schema change tools (pt-online-schema-change, gh-ost) prevent table locking during large-scale alterations. The database-per-service pattern enforces microservices boundaries, preventing tight coupling through shared database tables while requiring careful coordination of distributed transactions and eventual consistency patterns.

Rollback strategies distinguish between application-level rollbacks (safe when schemas are backward compatible) and database-level rollbacks (requiring undo migrations or point-in-time recovery from backups). CI/CD integration ensures migrations pass validation gates, execute in isolated staging environments, and trigger only after protective backups in production, with Kubernetes Jobs providing the execution context for schema changes within containerized environments.

**Key Takeaways:**
- Always use the expand-contract pattern for schema changes: add new columns/tables in one release (expand), migrate data and update applications, then remove old structures in subsequent releases (contract).
- Never drop or rename columns immediately; maintain backward compatibility so old application versions can run against new schemas during rolling deployments.
- Use `CREATE INDEX CONCURRENTLY` in PostgreSQL or online schema change tools (gh-ost, pt-online-schema-change) for large tables to prevent table locking and application downtime.
- Store migrations in version control alongside application code, but execute them via separate CI/CD jobs or Kubernetes Init Containers before application startup, using tools like Flyway or Liquibase for tracking and ordering.
- Implement idempotent migrations (safe to re-run) and always backup production databases before applying migrations, with automated rollback procedures tested in staging environments.

**Next Chapter Preview:**
Chapter 42: Infrastructure as Code in CI/CD explores managing cloud infrastructure through declarative configuration files versioned alongside application code. We will examine Terraform workflows for creating and modifying cloud resources (VPCs, databases, Kubernetes clusters) within CI/CD pipelines, handling state management and locking to prevent conflicts, implementing plan/apply stages for safe infrastructure changes, and strategies for drift detection when manual console changes occur. The chapter covers modular infrastructure design, secret management for cloud credentials, and integration with policy-as-code frameworks to ensure infrastructure compliance, extending the database and application deployment patterns to the underlying infrastructure layer.